In [11]:
# VERIFICACIÓN DESCARGA OHLCV DAILY
from pathlib import Path
import polars as pl
import random

# Config (rutas relativas desde 01_daily/01_agregation_OHLCV/)
UNIVERSE = "../../processed/universe/smallcaps_universe_2025-11-01.parquet"
DAILY_DIR = Path("../../raw/polygon/ohlcv_daily")

# 1. Universo vs Descargados
df_universe = pl.read_parquet(UNIVERSE)
universe_tickers = set(df_universe["ticker"].to_list())
downloaded = {d.name for d in DAILY_DIR.iterdir() if d.is_dir() and d.name != "_batch_temp"}

print(f"📊 RESUMEN DESCARGA DAILY")
print(f"{'='*50}")
print(f"Universo esperado:    {len(universe_tickers):,} tickers")
print(f"Descargados:          {len(downloaded):,} tickers")
print(f"Cobertura:            {len(downloaded)/len(universe_tickers)*100:.2f}%")
print(f"Faltantes:            {len(universe_tickers - downloaded):,}")

# 2. Sample 3 tickers (LEER TODOS LOS AÑOS)
print(f"\n📁 SAMPLE  TICKERS:")
print(f"{'='*50}")
for ticker in random.sample(sorted(downloaded), min(5, len(downloaded))):
    ticker_dir = DAILY_DIR / ticker
    years = sorted([y.name for y in ticker_dir.glob("year=*")])
    
    # Leer TODOS los años y concatenar
    dfs = []
    for year in years:
        df_year = pl.read_parquet(ticker_dir / year / "daily.parquet")
        dfs.append(df_year)
    
    df_all = pl.concat(dfs)
    dates = df_all["date"].sort()
    
    print(f"{ticker:8s} | {len(years)} años | {len(df_all):,} rows | {dates[0]} → {dates[-1]}")

# 3. Faltantes (si hay)
missing = universe_tickers - downloaded
if missing:
    print(f"\n❌ FALTANTES ({len(missing)}):")
    print(f"{'='*50}")
    print(f"Primeros 10: {sorted(list(missing))[:10]}")
else:
    print(f"\n✅ TODOS LOS TICKERS DESCARGADOS")

📊 RESUMEN DESCARGA DAILY
Universo esperado:    6,405 tickers
Descargados:          6,297 tickers
Cobertura:            98.31%
Faltantes:            108

📁 SAMPLE  TICKERS:
AFI      | 4 años | 854 rows | 2019-01-02 → 2022-05-20
RETA     | 5 años | 1,190 rows | 2019-01-02 → 2023-09-25
CMRE     | 7 años | 1,719 rows | 2019-01-02 → 2025-10-31
CWBC     | 7 años | 1,596 rows | 2019-01-02 → 2025-10-31
MGEN     | 3 años | 516 rows | 2019-01-02 → 2021-01-19

❌ FALTANTES (108):
Primeros 10: ['AANW', 'ABX', 'ACLL', 'AIRCW', 'AIRTV', 'AIVW', 'ALPX', 'ALVU', 'ARMKW', 'ARNCW']


In [12]:
# VERIFICACIÓN DESCARGA OHLCV INTRADÍA 1-MINUTE
from pathlib import Path
import polars as pl
import random

# Config
UNIVERSE = "../../processed/universe/smallcaps_universe_2025-11-01.parquet"
INTRADAY_DIR = Path("../../raw/polygon/ohlcv_intraday_1m")

# 1. Universo vs Descargados
df_universe = pl.read_parquet(UNIVERSE)
universe_tickers = set(df_universe["ticker"].to_list())
downloaded = {d.name for d in INTRADAY_DIR.iterdir() if d.is_dir() and d.name != "_batch_temp"}

print(f"📊 RESUMEN DESCARGA INTRADÍA 1-MINUTE")
print(f"{'='*60}")
print(f"Universo esperado:    {len(universe_tickers):,} tickers")
print(f"Descargados:          {len(downloaded):,} tickers")
print(f"Cobertura:            {len(downloaded)/len(universe_tickers)*100:.2f}%")
print(f"Faltantes:            {len(universe_tickers - downloaded):,}")

# 2. Sample 3 tickers (leer TODOS los meses)
print(f"\n📁 SAMPLE 5 TICKERS:")
print(f"{'='*60}")
for ticker in random.sample(sorted(downloaded), min(5, len(downloaded))):
    ticker_dir = INTRADAY_DIR / ticker
    years = sorted([y.name for y in ticker_dir.glob("year=*")])
    
    # Contar archivos y rows totales
    total_rows = 0
    total_months = 0
    for year_dir in ticker_dir.glob("year=*"):
        for month_dir in year_dir.glob("month=*"):
            parquet = month_dir / "minute.parquet"
            if parquet.exists():
                df = pl.read_parquet(parquet)
                total_rows += len(df)
                total_months += 1
    
    # Leer primer y último mes para fechas
    first_year = ticker_dir / years[0]
    last_year = ticker_dir / years[-1]
    
    first_month = sorted(first_year.glob("month=*"))[0] / "minute.parquet"
    last_month = sorted(last_year.glob("month=*"))[-1] / "minute.parquet"
    
    df_first = pl.read_parquet(first_month)
    df_last = pl.read_parquet(last_month)
    
    first_date = df_first["minute"].sort()[0]
    last_date = df_last["minute"].sort()[-1]
    
    print(f"{ticker:8s} | {len(years)} años | {total_months} meses | {total_rows:,} rows | {first_date} → {last_date}")

# 3. Faltantes (si hay)
missing = universe_tickers - downloaded
if missing:
    print(f"\n❌ FALTANTES ({len(missing)}):")
    print(f"{'='*60}")
    print(f"Primeros 10: {sorted(list(missing))[:10]}")
else:
    print(f"\n✅ TODOS LOS TICKERS DESCARGADOS")

# 4. Estimación de tamaño total
print(f"\n💾 ESTIMACIÓN TAMAÑO:")
print(f"{'='*60}")
sample_size = min(10, len(downloaded))
sample_tickers = random.sample(sorted(downloaded), sample_size)

total_size_bytes = 0
for ticker in sample_tickers:
    ticker_dir = INTRADAY_DIR / ticker
    for parquet in ticker_dir.glob("**/*.parquet"):
        total_size_bytes += parquet.stat().st_size

avg_size_mb = (total_size_bytes / sample_size) / (1024 * 1024)
estimated_total_gb = (avg_size_mb * len(downloaded)) / 1024

print(f"Sample {sample_size} tickers:")
print(f"  Promedio/ticker: {avg_size_mb:.1f} MB")
print(f"  Total estimado:  {estimated_total_gb:.1f} GB ({len(downloaded)} tickers)")

📊 RESUMEN DESCARGA INTRADÍA 1-MINUTE
Universo esperado:    6,405 tickers
Descargados:          6,296 tickers
Cobertura:            98.30%
Faltantes:            109

📁 SAMPLE 5 TICKERS:
OGN      | 5 años | 53 meses | 432,251 rows | 2021-06-03 08:37 → 2025-10-31 23:50
GNL      | 7 años | 82 meses | 565,204 rows | 2019-01-02 13:05 → 2025-10-31 20:31
SVCO     | 2 años | 18 meses | 59,272 rows | 2024-05-09 15:44 → 2025-10-31 20:26
PRG      | 6 años | 59 meses | 327,602 rows | 2020-12-01 14:34 → 2025-10-31 20:00
RSSS     | 6 años | 68 meses | 50,478 rows | 2020-03-23 17:17 → 2025-10-31 20:21

❌ FALTANTES (109):
Primeros 10: ['AANW', 'ABX', 'ACLL', 'AEBIV', 'AIRCW', 'AIRTV', 'AIVW', 'ALPX', 'ALVU', 'ARMKW']

💾 ESTIMACIÓN TAMAÑO:
Sample 10 tickers:
  Promedio/ticker: 3.3 MB
  Total estimado:  20.6 GB (6296 tickers)


In [13]:
import polars as pl
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

# =====================================================================
# CONFIGURACIÓN
# =====================================================================
UNIVERSE = Path("../../processed/universe/smallcaps_universe_2025-11-01.parquet")
DAILY_DIR = Path("../../raw/polygon/ohlcv_daily")
INTRADAY_DIR = Path("../../raw/polygon/ohlcv_intraday_1m")
TRADES_DIR = Path("../../raw/polygon/trades_ticks")

# Cargar universo de tickers disponibles
df_universe = pl.read_parquet(UNIVERSE)
available_tickers = sorted(df_universe["ticker"].to_list())

# =====================================================================
# FUNCIONES DE CARGA DE DATOS
# =====================================================================

def load_daily_data(ticker, year):
    """Carga datos OHLCV diarios para un ticker y año"""
    daily_path = DAILY_DIR / ticker / f"year={year}" / "daily.parquet"
    if not daily_path.exists():
        return None
    return pl.read_parquet(daily_path)

def load_intraday_data(ticker, year, month):
    """Carga datos OHLCV intradiarios 1-min para un ticker, año y mes"""
    intraday_path = INTRADAY_DIR / ticker / f"year={year}" / f"month={month:02d}" / "minute.parquet"
    if not intraday_path.exists():
        return None
    return pl.read_parquet(intraday_path)

def load_trades_data(ticker, date_str):
    """Carga datos de trades (premarket + market) para un ticker y fecha específica"""
    try:
        year, month = date_str.split("-")[0], date_str.split("-")[1]
        day_dir = TRADES_DIR / ticker / f"year={year}" / f"month={month}" / f"day={date_str}"
        
        if not day_dir.exists():
            return None, None
        
        premarket_path = day_dir / "premarket.parquet"
        market_path = day_dir / "market.parquet"
        
        df_pre = pl.read_parquet(premarket_path) if premarket_path.exists() else None
        df_mkt = pl.read_parquet(market_path) if market_path.exists() else None
        
        return df_pre, df_mkt
    except Exception as e:
        print(f"Error cargando trades: {e}")
        return None, None

# =====================================================================
# FUNCIONES DE GRAFICACIÓN
# =====================================================================

def plot_daily_candlestick(ticker, year):
    """Gráfico de velas diario"""
    df = load_daily_data(ticker, year)
    if df is None or df.is_empty():
        print(f"❌ No hay datos diarios para {ticker} en {year}")
        return
    
    fig = go.Figure(data=[go.Candlestick(
        x=df["date"],
        open=df["o"],
        high=df["h"],
        low=df["l"],
        close=df["c"],
        name="OHLC"
    )])
    
    fig.add_trace(go.Bar(
        x=df["date"],
        y=df["v"],
        name="Volume",
        yaxis="y2",
        marker_color='rgba(100,100,250,0.3)'
    ))
    
    fig.update_layout(
        title=f"{ticker} - Daily OHLCV ({year})",
        yaxis_title="Price ($)",
        yaxis2=dict(title="Volume", overlaying="y", side="right"),
        xaxis_rangeslider_visible=False,
        height=600,
        template="plotly_dark"
    )
    
    fig.show()
    print(f"✅ {len(df):,} bars | {df['date'].min()} → {df['date'].max()}")

def plot_intraday_candlestick(ticker, date_str):
    """Gráfico de velas intradiario 1-min para un día específico"""
    year, month = date_str.split("-")[0], int(date_str.split("-")[1])
    df = load_intraday_data(ticker, year, month)
    
    if df is None or df.is_empty():
        print(f"❌ No hay datos intradiarios para {ticker} en {year}-{month:02d}")
        return
    
    # Filtrar por fecha específica
    df = df.filter(pl.col("date") == date_str)
    
    if df.is_empty():
        print(f"❌ No hay datos intradiarios para {ticker} en {date_str}")
        return
    
    fig = go.Figure(data=[go.Candlestick(
        x=df["minute"],
        open=df["o"],
        high=df["h"],
        low=df["l"],
        close=df["c"],
        name="OHLC"
    )])
    
    fig.add_trace(go.Bar(
        x=df["minute"],
        y=df["v"],
        name="Volume",
        yaxis="y2",
        marker_color='rgba(100,100,250,0.3)'
    ))
    
    fig.update_layout(
        title=f"{ticker} - Intraday 1-min ({date_str})",
        yaxis_title="Price ($)",
        yaxis2=dict(title="Volume", overlaying="y", side="right"),
        xaxis_rangeslider_visible=False,
        height=600,
        template="plotly_dark"
    )
    
    fig.show()
    print(f"✅ {len(df):,} 1-min bars | {df['minute'].min()} → {df['minute'].max()}")

def plot_trades_tick(ticker, date_str):
    """Gráfico de trades tick-level (premarket + market)"""
    df_pre, df_mkt = load_trades_data(ticker, date_str)
    
    if df_pre is None and df_mkt is None:
        print(f"❌ No hay datos de trades para {ticker} en {date_str}")
        return
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=("Premarket (04:00-09:30 ET)", "Market (09:30-16:00 ET)"),
        row_heights=[0.5, 0.5],
        vertical_spacing=0.1
    )
    
    # Premarket
    if df_pre is not None and not df_pre.is_empty():
        fig.add_trace(go.Scatter(
            x=df_pre["timestamp"],
            y=df_pre["price"],
            mode='markers',
            marker=dict(size=3, color=df_pre["size"], colorscale='Blues', showscale=True),
            name="Premarket",
            text=[f"Size: {s}" for s in df_pre["size"].to_list()],
            hovertemplate='%{x}<br>Price: $%{y:.2f}<br>%{text}<extra></extra>'
        ), row=1, col=1)
        print(f"✅ Premarket: {len(df_pre):,} trades | {df_pre['timestamp'].min()} → {df_pre['timestamp'].max()}")
    else:
        print(f"⚠️  No premarket trades")
    
    # Market
    if df_mkt is not None and not df_mkt.is_empty():
        fig.add_trace(go.Scatter(
            x=df_mkt["timestamp"],
            y=df_mkt["price"],
            mode='markers',
            marker=dict(size=3, color=df_mkt["size"], colorscale='Reds', showscale=True),
            name="Market",
            text=[f"Size: {s}" for s in df_mkt["size"].to_list()],
            hovertemplate='%{x}<br>Price: $%{y:.2f}<br>%{text}<extra></extra>'
        ), row=2, col=1)
        print(f"✅ Market: {len(df_mkt):,} trades | {df_mkt['timestamp'].min()} → {df_mkt['timestamp'].max()}")
    else:
        print(f"⚠️  No market trades")
    
    fig.update_layout(
        title=f"{ticker} - Trades Tick-Level ({date_str})",
        height=800,
        showlegend=True,
        template="plotly_dark"
    )
    
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Price ($)", row=2, col=1)
    
    fig.show()

# =====================================================================
# INTERFAZ INTERACTIVA
# =====================================================================

def create_interactive_chart():
    """Crea widgets interactivos para selección de ticker y tipo de gráfico"""
    
    # Widgets
    ticker_dropdown = widgets.Dropdown(
        options=available_tickers,
        value=available_tickers[0] if available_tickers else None,
        description='Ticker:',
        style={'description_width': '100px'}
    )
    
    chart_type = widgets.RadioButtons(
        options=['Daily', 'Intraday 1-min', 'Trades Tick-Level'],
        value='Daily',
        description='Tipo:',
        style={'description_width': '100px'}
    )
    
    date_picker = widgets.DatePicker(
        description='Fecha:',
        value=datetime(2025, 10, 31),
        style={'description_width': '100px'}
    )
    
    year_dropdown = widgets.Dropdown(
        options=list(range(2019, 2026)),
        value=2025,
        description='Año:',
        style={'description_width': '100px'}
    )
    
    plot_button = widgets.Button(
        description='📊 Graficar',
        button_style='success',
        tooltip='Click para generar gráfico'
    )
    
    output = widgets.Output()
    
    def on_button_click(b):
        with output:
            output.clear_output()
            ticker = ticker_dropdown.value
            chart = chart_type.value
            
            if chart == 'Daily':
                year = year_dropdown.value
                plot_daily_candlestick(ticker, year)
            
            elif chart == 'Intraday 1-min':
                date_str = date_picker.value.strftime("%Y-%m-%d")
                plot_intraday_candlestick(ticker, date_str)
            
            elif chart == 'Trades Tick-Level':
                date_str = date_picker.value.strftime("%Y-%m-%d")
                plot_trades_tick(ticker, date_str)
    
    plot_button.on_click(on_button_click)
    
    # Layout
    controls = widgets.VBox([
        widgets.HBox([ticker_dropdown, chart_type]),
        widgets.HBox([year_dropdown, date_picker]),
        plot_button
    ])
    
    display(controls, output)

# =====================================================================
# EJECUTAR
# =====================================================================
create_interactive_chart()

Output()

In [4]:
# ============================================================================
# VISUALIZACION TRADES TICK-LEVEL - TSIS SmallCaps
# ============================================================================

from pathlib import Path
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display

# CONFIGURACION - Path absoluto
TRADES_DIR = Path(r"D:\TSIS_SmallCaps\raw\polygon\trades_ticks")

# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================

def get_available_tickers():
    """Obtiene lista de tickers con datos"""
    if not TRADES_DIR.exists():
        print(f"ERROR: El directorio {TRADES_DIR} no existe")
        return []
    
    tickers = []
    for ticker_dir in TRADES_DIR.iterdir():
        if ticker_dir.is_dir() and ticker_dir.name != '_batch_temp':
            tickers.append(ticker_dir.name)
    return sorted(tickers)

def get_available_dates(ticker):
    """Obtiene fechas disponibles para un ticker"""
    ticker_dir = TRADES_DIR / ticker
    dates = set()
    
    for day_dir in ticker_dir.rglob('day=*'):
        date_str = day_dir.name.replace('day=', '')
        dates.add(date_str)
    
    return sorted(list(dates))

def load_ticker_day(ticker, date):
    """Carga trades de un día completo (premarket + market)"""
    year, month, _ = date.split('-')
    day_dir = TRADES_DIR / ticker / f"year={year}" / f"month={month}" / f"day={date}"
    
    dfs = []
    
    # Cargar premarket
    pre_file = day_dir / "premarket.parquet"
    if pre_file.exists():
        df_pre = pl.read_parquet(pre_file).with_columns(
            pl.lit("premarket").alias("session")
        )
        dfs.append(df_pre)
    
    # Cargar market
    mkt_file = day_dir / "market.parquet"
    if mkt_file.exists():
        df_mkt = pl.read_parquet(mkt_file).with_columns(
            pl.lit("market").alias("session")
        )
        dfs.append(df_mkt)
    
    if dfs:
        return pl.concat(dfs, how="vertical_relaxed").sort("timestamp")
    return None

def aggregate_to_ohlc(df, interval="1m"):
    """Agrega trades tick-level a barras OHLC"""
    if df is None or df.height == 0:
        return None
    
    # Truncar timestamp al intervalo especificado
    df_agg = df.with_columns(
        pl.col("timestamp").dt.truncate(interval).alias("bar_time")
    ).group_by(["bar_time", "session"]).agg([
        pl.col("price").first().alias("open"),
        pl.col("price").max().alias("high"),
        pl.col("price").min().alias("low"),
        pl.col("price").last().alias("close"),
        pl.col("size").sum().alias("volume"),
        pl.count().alias("trades")
    ]).sort("bar_time")
    
    return df_agg

# ============================================================================
# GRAFICO 1: DIA COMPLETO CON CANDLESTICKS
# ============================================================================

def plot_day_ohlc(ticker, date, interval="1m"):
    """Grafica día completo con candlesticks a partir de trades"""
    df_trades = load_ticker_day(ticker, date)
    
    if df_trades is None or df_trades.height == 0:
        print(f"No hay datos para {ticker} en {date}")
        return
    
    # Agregar a OHLC
    df_ohlc = aggregate_to_ohlc(df_trades, interval)
    
    if df_ohlc is None:
        print(f"No se pudo agregar datos a OHLC")
        return
    
    # Crear figura con subplots
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            f'{ticker} - Candlestick {interval} ({date})',
            'Volumen',
            'Número de Trades por Barra'
        ),
        row_heights=[0.5, 0.25, 0.25]
    )
    
    # Separar por sesión
    df_pre = df_ohlc.filter(pl.col("session") == "premarket")
    df_mkt = df_ohlc.filter(pl.col("session") == "market")
    
    # Plot 1: Candlesticks - Premarket
    if df_pre.height > 0:
        fig.add_trace(
            go.Candlestick(
                x=df_pre["bar_time"],
                open=df_pre["open"],
                high=df_pre["high"],
                low=df_pre["low"],
                close=df_pre["close"],
                name='Premarket',
                increasing_line_color='darkorange',
                decreasing_line_color='orange'
            ),
            row=1, col=1
        )
    
    # Plot 1: Candlesticks - Market
    if df_mkt.height > 0:
        fig.add_trace(
            go.Candlestick(
                x=df_mkt["bar_time"],
                open=df_mkt["open"],
                high=df_mkt["high"],
                low=df_mkt["low"],
                close=df_mkt["close"],
                name='Market',
                increasing_line_color='green',
                decreasing_line_color='red'
            ),
            row=1, col=1
        )
    
    # Plot 2: Volumen
    for session, color in [("premarket", "orange"), ("market", "blue")]:
        df_sess = df_ohlc.filter(pl.col("session") == session)
        if df_sess.height > 0:
            fig.add_trace(
                go.Bar(
                    x=df_sess["bar_time"],
                    y=df_sess["volume"],
                    name=f'Vol {session}',
                    marker_color=color,
                    opacity=0.6,
                    showlegend=False
                ),
                row=2, col=1
            )
    
    # Plot 3: Número de trades
    for session, color in [("premarket", "orange"), ("market", "blue")]:
        df_sess = df_ohlc.filter(pl.col("session") == session)
        if df_sess.height > 0:
            fig.add_trace(
                go.Bar(
                    x=df_sess["bar_time"],
                    y=df_sess["trades"],
                    name=f'Trades {session}',
                    marker_color=color,
                    opacity=0.6,
                    showlegend=False
                ),
                row=3, col=1
            )
    
    # Actualizar layout
    fig.update_xaxes(title_text="Timestamp (UTC)", row=3, col=1)
    fig.update_yaxes(title_text="Precio ($)", row=1, col=1)
    fig.update_yaxes(title_text="Volumen", row=2, col=1)
    fig.update_yaxes(title_text="# Trades", row=3, col=1)
    
    fig.update_layout(
        height=900,
        title_text=f"{ticker} - OHLC {interval}: {date}",
        xaxis_rangeslider_visible=False
    )
    
    # Estadísticas
    print(f"\n{'='*60}")
    print(f"ESTADISTICAS: {ticker} - {date} (Agregado a {interval})")
    print(f"{'='*60}")
    print(f"Total barras:        {df_ohlc.height}")
    print(f"  Premarket:         {df_pre.height}")
    print(f"  Market:            {df_mkt.height}")
    print(f"Total trades:        {df_trades.height:,}")
    print(f"Precio min:          ${df_ohlc['low'].min():.4f}")
    print(f"Precio max:          ${df_ohlc['high'].max():.4f}")
    print(f"Volumen total:       {df_ohlc['volume'].sum():,}")
    print(f"Primera barra:       {df_ohlc['bar_time'][0]}")
    print(f"Ultima barra:        {df_ohlc['bar_time'][-1]}")
    print(f"{'='*60}\n")
    
    fig.show()

# ============================================================================
# GRAFICO 2: MULTIPLES DIAS
# ============================================================================

def plot_ticker_multiday(ticker, max_days=10, interval="5m"):
    """Grafica múltiples días en candlesticks"""
    ticker_dir = TRADES_DIR / ticker
    
    # Cargar últimos N días
    all_dfs = []
    dates_loaded = []
    
    for day_dir in sorted(ticker_dir.rglob('day=*'), reverse=True)[:max_days]:
        date_str = day_dir.name.replace('day=', '')
        year, month = date_str.split('-')[:2]
        
        dfs = []
        for session_file, session_name in [
            (day_dir / "premarket.parquet", "premarket"),
            (day_dir / "market.parquet", "market")
        ]:
            if session_file.exists():
                df = pl.read_parquet(session_file).with_columns(
                    pl.lit(session_name).alias("session")
                )
                dfs.append(df)
        
        if dfs:
            day_df = pl.concat(dfs, how="vertical_relaxed")
            all_dfs.append(day_df)
            dates_loaded.append(date_str)
    
    if not all_dfs:
        print(f"No hay datos para {ticker}")
        return
    
    # Combinar todos los días
    df_trades = pl.concat(all_dfs, how="vertical_relaxed").sort("timestamp")
    df_ohlc = aggregate_to_ohlc(df_trades, interval)
    
    if df_ohlc is None:
        print("No se pudo agregar datos")
        return
    
    # Crear figura
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            f'{ticker} - Últimos {len(dates_loaded)} días',
            'Volumen'
        ),
        row_heights=[0.7, 0.3]
    )
    
    # Separar por sesión
    df_pre = df_ohlc.filter(pl.col("session") == "premarket")
    df_mkt = df_ohlc.filter(pl.col("session") == "market")
    
    # Candlesticks
    if df_pre.height > 0:
        fig.add_trace(
            go.Candlestick(
                x=df_pre["bar_time"],
                open=df_pre["open"],
                high=df_pre["high"],
                low=df_pre["low"],
                close=df_pre["close"],
                name='Premarket',
                increasing_line_color='darkorange',
                decreasing_line_color='orange'
            ),
            row=1, col=1
        )
    
    if df_mkt.height > 0:
        fig.add_trace(
            go.Candlestick(
                x=df_mkt["bar_time"],
                open=df_mkt["open"],
                high=df_mkt["high"],
                low=df_mkt["low"],
                close=df_mkt["close"],
                name='Market',
                increasing_line_color='green',
                decreasing_line_color='red'
            ),
            row=1, col=1
        )
    
    # Volumen
    for session, color in [("premarket", "orange"), ("market", "blue")]:
        df_sess = df_ohlc.filter(pl.col("session") == session)
        if df_sess.height > 0:
            fig.add_trace(
                go.Bar(
                    x=df_sess["bar_time"],
                    y=df_sess["volume"],
                    name=f'Vol {session}',
                    marker_color=color,
                    opacity=0.6,
                    showlegend=False
                ),
                row=2, col=1
            )
    
    fig.update_xaxes(title_text="Timestamp (UTC)", row=2, col=1)
    fig.update_yaxes(title_text="Precio ($)", row=1, col=1)
    fig.update_yaxes(title_text="Volumen", row=2, col=1)
    
    fig.update_layout(
        height=800,
        title_text=f"{ticker} - Últimos {len(dates_loaded)} días (OHLC {interval})",
        xaxis_rangeslider_visible=False
    )
    
    print(f"\n{'='*60}")
    print(f"RESUMEN: {ticker} - Últimos {len(dates_loaded)} días")
    print(f"{'='*60}")
    print(f"Fechas: {dates_loaded[-1]} a {dates_loaded[0]}")
    print(f"Total barras {interval}:  {df_ohlc.height}")
    print(f"Total trades:        {df_trades.height:,}")
    print(f"Precio min:          ${df_ohlc['low'].min():.4f}")
    print(f"Precio max:          ${df_ohlc['high'].max():.4f}")
    print(f"Volumen total:       {df_ohlc['volume'].sum():,}")
    print(f"{'='*60}\n")
    
    fig.show()

# ============================================================================
# WIDGETS INTERACTIVOS
# ============================================================================

def create_interactive_charts():
    """Crea widgets interactivos para explorar los datos"""
    
    # Obtener tickers disponibles
    available_tickers = get_available_tickers()
    
    if not available_tickers:
        print("No hay tickers disponibles")
        return
    
    # Widgets
    ticker_dropdown = widgets.Dropdown(
        options=available_tickers,
        description='Ticker:',
        value=available_tickers[0]
    )
    
    chart_type = widgets.RadioButtons(
        options=['Un Día', 'Múltiples Días'],
        description='Tipo:',
        value='Un Día'
    )
    
    date_dropdown = widgets.Dropdown(
        options=[],
        description='Fecha:',
        disabled=False
    )
    
    interval_dropdown = widgets.Dropdown(
        options=['1m', '5m', '15m', '30m', '1h'],
        description='Intervalo:',
        value='1m'
    )
    
    max_days_slider = widgets.IntSlider(
        value=10,
        min=5,
        max=30,
        step=5,
        description='Días:',
        continuous_update=False,
        layout=widgets.Layout(visibility='hidden')
    )
    
    plot_button = widgets.Button(
        description='Graficar',
        button_style='primary',
        icon='chart-line'
    )
    
    output = widgets.Output()
    
    # Callbacks
    def update_dates(change):
        ticker = ticker_dropdown.value
        if not ticker:
            return
        dates = get_available_dates(ticker)
        date_dropdown.options = dates if dates else ['Sin datos']
        if dates:
            date_dropdown.value = dates[-1]
    
    def on_chart_type_change(change):
        if change['new'] == 'Un Día':
            date_dropdown.layout.visibility = 'visible'
            date_dropdown.disabled = False
            max_days_slider.layout.visibility = 'hidden'
        else:
            date_dropdown.layout.visibility = 'hidden'
            date_dropdown.disabled = True
            max_days_slider.layout.visibility = 'visible'
    
    def on_plot_click(b):
        with output:
            output.clear_output(wait=True)
            
            try:
                if chart_type.value == 'Un Día':
                    if date_dropdown.value and date_dropdown.value != 'Sin datos':
                        plot_day_ohlc(ticker_dropdown.value, date_dropdown.value, interval_dropdown.value)
                    else:
                        print("No hay fechas disponibles para este ticker")
                else:
                    plot_ticker_multiday(ticker_dropdown.value, max_days_slider.value, interval_dropdown.value)
            except Exception as e:
                print(f"Error: {e}")
                import traceback
                traceback.print_exc()
    
    # Conectar callbacks
    ticker_dropdown.observe(update_dates, 'value')
    chart_type.observe(on_chart_type_change, 'value')
    plot_button.on_click(on_plot_click)
    
    # Inicializar
    update_dates(None)
    
    # Display
    display(widgets.VBox([
        ticker_dropdown,
        chart_type,
        date_dropdown,
        interval_dropdown,
        max_days_slider,
        plot_button,
        output
    ]))

# ============================================================================
# EJECUTAR
# ============================================================================

print("Visualización Trades Tick-Level (OHLC) - TSIS SmallCaps")
print("=" * 60)
print(f"Directorio: {TRADES_DIR}")
print(f"Tickers disponibles: {len(get_available_tickers())}")
print("=" * 60)
print("\nUso:")
print("  create_interactive_charts()  - Interfaz interactiva")
print("  plot_day_ohlc('DRUG', '2025-01-02', '1m')  - Día con candlesticks")
print("  plot_ticker_multiday('DRUG', max_days=10, interval='5m')  - Múltiples días")
print()

# Crear interfaz interactiva
create_interactive_charts()

Visualización Trades Tick-Level (OHLC) - TSIS SmallCaps
Directorio: D:\TSIS_SmallCaps\raw\polygon\trades_ticks
Tickers disponibles: 40

Uso:
  create_interactive_charts()  - Interfaz interactiva
  plot_day_ohlc('DRUG', '2025-01-02', '1m')  - Día con candlesticks
  plot_ticker_multiday('DRUG', max_days=10, interval='5m')  - Múltiples días



In [2]:
# ========================================================================
# VISUALIZADOR INTERACTIVO DE TRADES TICK-LEVEL (CORREGIDO)
# ========================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pytz
from pathlib import Path

# Zona horaria
ET = pytz.timezone('America/New_York')

# Rutas
TRADES_DIR = Path(r"D:\TSIS_SmallCaps\raw\polygon\trades_ticks")

# =====================================================================
# FUNCIÓN PRINCIPAL DE VISUALIZACIÓN
# =====================================================================

def plot_trades_analysis(ticker, date_str):
    """
    Visualiza trades tick-level con 2 paneles:
    1. Premarket trades (04:00-09:30 ET)
    2. Market trades (09:30-20:00 ET)
    """
    
    try:
        year, month = date_str.split("-")[0], date_str.split("-")[1]
        day_dir = TRADES_DIR / ticker / f"year={year}" / f"month={month}" / f"day={date_str}"
        
        if not day_dir.exists():
            print(f"❌ No se encontró directorio: {day_dir}")
            return None
        
        premarket_path = day_dir / "premarket.parquet"
        market_path = day_dir / "market.parquet"
        
        # Cargar datos
        df_pre = None
        df_mkt = None
        
        if premarket_path.exists():
            df_pre = pl.read_parquet(premarket_path)
            # ✅ FIX: El timestamp YA es Datetime, solo necesita timezone
            df_pre = df_pre.with_columns([
                pl.col("timestamp")
                  .dt.replace_time_zone("UTC")
                  .dt.convert_time_zone("America/New_York")
                  .alias("timestamp_et")
            ])
        
        if market_path.exists():
            df_mkt = pl.read_parquet(market_path)
            # ✅ FIX: El timestamp YA es Datetime, solo necesita timezone
            df_mkt = df_mkt.with_columns([
                pl.col("timestamp")
                  .dt.replace_time_zone("UTC")
                  .dt.convert_time_zone("America/New_York")
                  .alias("timestamp_et")
            ])
        
        if df_pre is None and df_mkt is None:
            print(f"❌ No hay datos de trades para {ticker} en {date_str}")
            return None
        
        # ================================================================
        # CREAR FIGURA CON 2 PANELES
        # ================================================================
        
        fig, axes = plt.subplots(2, 1, figsize=(20, 12))
        
        # ----------------------------------------------------------------
        # PANEL 1: PREMARKET TRADES
        # ----------------------------------------------------------------
        if df_pre is not None and len(df_pre) > 0:
            df_pre_pd = df_pre.to_pandas()
            
            # 🎨 LÍNEA CONTINUA para ver el movimiento del precio
            axes[0].plot(
                df_pre_pd['timestamp_et'],
                df_pre_pd['price'],
                color='steelblue',
                linewidth=1,
                alpha=0.5,
                zorder=1
            )
            
            # 🔵 SCATTER con tamaño proporcional al volumen
            scatter = axes[0].scatter(
                df_pre_pd['timestamp_et'],
                df_pre_pd['price'],
                c=df_pre_pd['size'],
                s=df_pre_pd['size'] / df_pre_pd['size'].max() * 100 + 10,
                cmap='Blues',
                alpha=0.7,
                edgecolors='navy',
                linewidth=0.5,
                zorder=2
            )
            
            # Colorbar
            cbar = plt.colorbar(scatter, ax=axes[0])
            cbar.set_label('Trade Size', fontsize=12, fontweight='bold')
            
            axes[0].set_ylabel('Precio ($)', fontsize=13, fontweight='bold')
            axes[0].set_title(
                f'{ticker} - PREMARKET Trades (04:00-09:30 ET) | {date_str} | {len(df_pre):,} trades',
                fontsize=15, fontweight='bold', pad=20
            )
            axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=ET))
            axes[0].xaxis.set_major_locator(mdates.MinuteLocator(interval=30))
            axes[0].grid(True, alpha=0.3, linestyle='--')
            plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')
            
            # Línea de precio promedio
            avg_price = df_pre_pd['price'].mean()
            axes[0].axhline(avg_price, color='orange', linestyle='--', 
                           linewidth=1.5, alpha=0.7, label=f'Avg: ${avg_price:.2f}')
            axes[0].legend(loc='upper left', fontsize=11)
            
            print(f"✅ PREMARKET: {len(df_pre):,} trades")
            print(f"   Timerange: {df_pre_pd['timestamp_et'].min()} → {df_pre_pd['timestamp_et'].max()}")
            print(f"   Price range: ${df_pre_pd['price'].min():.2f} → ${df_pre_pd['price'].max():.2f}")
            print(f"   Total volume: {df_pre_pd['size'].sum():,}")
        else:
            axes[0].text(0.5, 0.5, 'Sin datos premarket', 
                        ha='center', va='center', fontsize=20, color='gray')
            axes[0].set_title(f'{ticker} - PREMARKET (sin datos)', fontsize=15, fontweight='bold')
            print(f"⚠️  No premarket trades")
        
        # ----------------------------------------------------------------
        # PANEL 2: MARKET TRADES
        # ----------------------------------------------------------------
        if df_mkt is not None and len(df_mkt) > 0:
            df_mkt_pd = df_mkt.to_pandas()
            
            # 🎨 LÍNEA CONTINUA para ver el movimiento del precio
            axes[1].plot(
                df_mkt_pd['timestamp_et'],
                df_mkt_pd['price'],
                color='darkred',
                linewidth=1,
                alpha=0.4,
                zorder=1
            )
            
            # 🔴 SCATTER con tamaño proporcional al volumen
            scatter = axes[1].scatter(
                df_mkt_pd['timestamp_et'],
                df_mkt_pd['price'],
                c=df_mkt_pd['size'],
                s=df_mkt_pd['size'] / df_mkt_pd['size'].max() * 100 + 10,
                cmap='Reds',
                alpha=0.7,
                edgecolors='darkred',
                linewidth=0.5,
                zorder=2
            )
            
            # Colorbar
            cbar = plt.colorbar(scatter, ax=axes[1])
            cbar.set_label('Trade Size', fontsize=12, fontweight='bold')
            
            axes[1].set_ylabel('Precio ($)', fontsize=13, fontweight='bold')
            axes[1].set_xlabel('Hora (ET - Eastern Time)', fontsize=13, fontweight='bold')
            axes[1].set_title(
                f'{ticker} - MARKET Trades (09:30-20:00 ET) | {date_str} | {len(df_mkt):,} trades',
                fontsize=15, fontweight='bold', pad=20
            )
            axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=ET))
            axes[1].xaxis.set_major_locator(mdates.HourLocator(interval=1, tz=ET))
            axes[1].grid(True, alpha=0.3, linestyle='--')
            plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
            
            # Línea de precio promedio
            avg_price = df_mkt_pd['price'].mean()
            axes[1].axhline(avg_price, color='orange', linestyle='--', 
                           linewidth=1.5, alpha=0.7, label=f'Avg: ${avg_price:.2f}')
            axes[1].legend(loc='upper left', fontsize=11)
            
            print(f"\n✅ MARKET: {len(df_mkt):,} trades")
            print(f"   Timerange: {df_mkt_pd['timestamp_et'].min()} → {df_mkt_pd['timestamp_et'].max()}")
            print(f"   Price range: ${df_mkt_pd['price'].min():.2f} → ${df_mkt_pd['price'].max():.2f}")
            print(f"   Total volume: {df_mkt_pd['size'].sum():,}")
        else:
            axes[1].text(0.5, 0.5, 'Sin datos market', 
                        ha='center', va='center', fontsize=20, color='gray')
            axes[1].set_title(f'{ticker} - MARKET (sin datos)', fontsize=15, fontweight='bold')
            axes[1].set_xlabel('Hora (ET)', fontsize=13, fontweight='bold')
            print(f"\n⚠️  No market trades")
        
        plt.tight_layout()
        plt.show()
        
        # ================================================================
        # ESTADÍSTICAS RESUMIDAS
        # ================================================================
        print(f"\n{'='*80}")
        print(f"📊 RESUMEN - {ticker} | {date_str}")
        print(f"{'='*80}")
        
        if df_pre is not None and len(df_pre) > 0:
            print(f"\n🌅 PREMARKET:")
            print(f"   Trades: {len(df_pre):,}")
            print(f"   Volume: {df_pre['size'].sum():,}")
            print(f"   Avg trade size: {df_pre['size'].mean():.1f}")
            vwap = (df_pre['price'] * df_pre['size']).sum() / df_pre['size'].sum()
            print(f"   Price VWAP: ${vwap:.2f}")
        
        if df_mkt is not None and len(df_mkt) > 0:
            print(f"\n📈 MARKET:")
            print(f"   Trades: {len(df_mkt):,}")
            print(f"   Volume: {df_mkt['size'].sum():,}")
            print(f"   Avg trade size: {df_mkt['size'].mean():.1f}")
            vwap = (df_mkt['price'] * df_mkt['size']).sum() / df_mkt['size'].sum()
            print(f"   Price VWAP: ${vwap:.2f}")
        
        print(f"\n{'='*80}")
        
        return df_pre, df_mkt
    
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# =====================================================================
# WIDGETS INTERACTIVOS
# =====================================================================

# Obtener tickers disponibles
ticker_dirs = [d.name for d in TRADES_DIR.iterdir() if d.is_dir() and d.name != '_batch_temp']
ticker_dirs = sorted(ticker_dirs)

print("="*80)
print("VISUALIZADOR INTERACTIVO DE TRADES TICK-LEVEL")
print("="*80)
print(f"\n✅ {len(ticker_dirs):,} tickers disponibles")
print(f"✅ Zona horaria: America/New_York (Eastern Time)")
print("\n")

# Crear widgets
ticker_dropdown = widgets.Dropdown(
    options=ticker_dirs,
    description='Ticker:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

date_dropdown = widgets.Dropdown(
    description='Fecha:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

plot_button = widgets.Button(
    description='📊 GRAFICAR TRADES',
    button_style='success',
    layout=widgets.Layout(width='250px', height='40px')
)

output_area = widgets.Output()

# =====================================================================
# LÓGICA DE ACTUALIZACIÓN
# =====================================================================

def update_dates(change):
    """Actualiza las fechas disponibles cuando se selecciona un ticker"""
    ticker = change['new']
    ticker_dir = TRADES_DIR / ticker
    
    # Buscar todas las fechas disponibles
    dates = []
    for year_dir in sorted(ticker_dir.glob("year=*")):
        for month_dir in sorted(year_dir.glob("month=*")):
            for day_dir in sorted(month_dir.glob("day=*")):
                date_str = day_dir.name.replace("day=", "")
                dates.append(date_str)
    
    date_dropdown.options = sorted(dates, reverse=True) if dates else ['Sin datos']
    if dates:
        date_dropdown.value = sorted(dates, reverse=True)[0]  # Fecha más reciente

def plot_selected(button):
    """Ejecuta la visualización cuando se presiona el botón"""
    with output_area:
        clear_output(wait=True)
        
        ticker = ticker_dropdown.value
        date_str = date_dropdown.value
        
        if date_str == 'Sin datos':
            print(f"❌ No hay datos disponibles para {ticker}")
            return
        
        print(f"🚀 Graficando trades tick-level: {ticker} | {date_str}")
        print("="*80)
        print()
        
        plot_trades_analysis(ticker=ticker, date_str=date_str)

# Conectar eventos
ticker_dropdown.observe(update_dates, names='value')
plot_button.on_click(plot_selected)

# Inicializar
if len(ticker_dirs) > 0:
    ticker_dropdown.value = ticker_dirs[0]

# =====================================================================
# MOSTRAR UI
# =====================================================================

print("📋 INSTRUCCIONES:")
print("1. Selecciona un TICKER (actualización automática de fechas)")
print("2. Selecciona una FECHA (ordenadas de más reciente a más antigua)")
print("3. Click en 'GRAFICAR TRADES'")
print("\n✨ Se mostrarán: Premarket + Market trades")
print("⏰ Todas las horas en ET (Eastern Time - Nueva York)")
print("🔵 Premarket: línea azul + puntos | 🔴 Market: línea roja + puntos")
print("📏 Tamaño del punto = tamaño del trade")
print("📈 Línea conecta los ticks para ver el movimiento del precio\n")

display(widgets.VBox([
    widgets.HBox([ticker_dropdown, date_dropdown]),
    plot_button,
    output_area
]))

VISUALIZADOR INTERACTIVO DE TRADES TICK-LEVEL

✅ 171 tickers disponibles
✅ Zona horaria: America/New_York (Eastern Time)


📋 INSTRUCCIONES:
1. Selecciona un TICKER (actualización automática de fechas)
2. Selecciona una FECHA (ordenadas de más reciente a más antigua)
3. Click en 'GRAFICAR TRADES'

✨ Se mostrarán: Premarket + Market trades
⏰ Todas las horas en ET (Eastern Time - Nueva York)
🔵 Premarket: línea azul + puntos | 🔴 Market: línea roja + puntos
📏 Tamaño del punto = tamaño del trade
📈 Línea conecta los ticks para ver el movimiento del precio



In [ ]:
# ========================================================================
# VISUALIZADOR INTERACTIVO DE TRADES TICK-LEVEL (CORREGIDO)
# ========================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pytz
from pathlib import Path

# Zona horaria
ET = pytz.timezone('America/New_York')

# Rutas
TRADES_DIR = Path(r"D:\TSIS_SmallCaps\raw\polygon\trades_ticks")

# =====================================================================
# FUNCIÓN PRINCIPAL DE VISUALIZACIÓN
# =====================================================================

def plot_trades_analysis(ticker, date_str):
    """
    Visualiza trades tick-level con 2 paneles:
    1. Premarket trades (04:00-09:30 ET)
    2. Market trades (09:30-20:00 ET)
    """
    
    try:
        year, month = date_str.split("-")[0], date_str.split("-")[1]
        day_dir = TRADES_DIR / ticker / f"year={year}" / f"month={month}" / f"day={date_str}"
        
        if not day_dir.exists():
            print(f"❌ No se encontró directorio: {day_dir}")
            return None
        
        premarket_path = day_dir / "premarket.parquet"
        market_path = day_dir / "market.parquet"
        
        # Cargar datos
        df_pre = None
        df_mkt = None
        
        if premarket_path.exists():
            df_pre = pl.read_parquet(premarket_path)
            # ✅ FIX: El timestamp YA es Datetime, solo necesita timezone
            df_pre = df_pre.with_columns([
                pl.col("timestamp")
                  .dt.replace_time_zone("UTC")
                  .dt.convert_time_zone("America/New_York")
                  .alias("timestamp_et")
            ])
        
        if market_path.exists():
            df_mkt = pl.read_parquet(market_path)
            # ✅ FIX: El timestamp YA es Datetime, solo necesita timezone
            df_mkt = df_mkt.with_columns([
                pl.col("timestamp")
                  .dt.replace_time_zone("UTC")
                  .dt.convert_time_zone("America/New_York")
                  .alias("timestamp_et")
            ])
        
        if df_pre is None and df_mkt is None:
            print(f"❌ No hay datos de trades para {ticker} en {date_str}")
            return None
        
        # ================================================================
        # CREAR FIGURA CON 2 PANELES
        # ================================================================
        
        fig, axes = plt.subplots(2, 1, figsize=(20, 12))
        
        # ----------------------------------------------------------------
        # PANEL 1: PREMARKET TRADES
        # ----------------------------------------------------------------
        if df_pre is not None and len(df_pre) > 0:
            df_pre_pd = df_pre.to_pandas()
            
            # 🎨 LÍNEA CONTINUA para ver el movimiento del precio
            axes[0].plot(
                df_pre_pd['timestamp_et'],
                df_pre_pd['price'],
                color='steelblue',
                linewidth=1,
                alpha=0.5,
                zorder=1
            )
            
            # 🔵 SCATTER con tamaño proporcional al volumen
            scatter = axes[0].scatter(
                df_pre_pd['timestamp_et'],
                df_pre_pd['price'],
                c=df_pre_pd['size'],
                s=df_pre_pd['size'] / df_pre_pd['size'].max() * 100 + 10,
                cmap='Blues',
                alpha=0.7,
                edgecolors='navy',
                linewidth=0.5,
                zorder=2
            )
            
            # Colorbar
            cbar = plt.colorbar(scatter, ax=axes[0])
            cbar.set_label('Trade Size', fontsize=12, fontweight='bold')
            
            axes[0].set_ylabel('Precio ($)', fontsize=13, fontweight='bold')
            axes[0].set_title(
                f'{ticker} - PREMARKET Trades (04:00-09:30 ET) | {date_str} | {len(df_pre):,} trades',
                fontsize=15, fontweight='bold', pad=20
            )
            axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=ET))
            axes[0].xaxis.set_major_locator(mdates.MinuteLocator(interval=30))
            axes[0].grid(True, alpha=0.3, linestyle='--')
            plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')
            
            # Línea de precio promedio
            avg_price = df_pre_pd['price'].mean()
            axes[0].axhline(avg_price, color='orange', linestyle='--', 
                           linewidth=1.5, alpha=0.7, label=f'Avg: ${avg_price:.2f}')
            axes[0].legend(loc='upper left', fontsize=11)
            
            print(f"✅ PREMARKET: {len(df_pre):,} trades")
            print(f"   Timerange: {df_pre_pd['timestamp_et'].min()} → {df_pre_pd['timestamp_et'].max()}")
            print(f"   Price range: ${df_pre_pd['price'].min():.2f} → ${df_pre_pd['price'].max():.2f}")
            print(f"   Total volume: {df_pre_pd['size'].sum():,}")
        else:
            axes[0].text(0.5, 0.5, 'Sin datos premarket', 
                        ha='center', va='center', fontsize=20, color='gray')
            axes[0].set_title(f'{ticker} - PREMARKET (sin datos)', fontsize=15, fontweight='bold')
            print(f"⚠️  No premarket trades")
        
        # ----------------------------------------------------------------
        # PANEL 2: MARKET TRADES
        # ----------------------------------------------------------------
        if df_mkt is not None and len(df_mkt) > 0:
            df_mkt_pd = df_mkt.to_pandas()
            
            # 🎨 LÍNEA CONTINUA para ver el movimiento del precio
            axes[1].plot(
                df_mkt_pd['timestamp_et'],
                df_mkt_pd['price'],
                color='darkred',
                linewidth=1,
                alpha=0.4,
                zorder=1
            )
            
            # 🔴 SCATTER con tamaño proporcional al volumen
            scatter = axes[1].scatter(
                df_mkt_pd['timestamp_et'],
                df_mkt_pd['price'],
                c=df_mkt_pd['size'],
                s=df_mkt_pd['size'] / df_mkt_pd['size'].max() * 100 + 10,
                cmap='Reds',
                alpha=0.7,
                edgecolors='darkred',
                linewidth=0.5,
                zorder=2
            )
            
            # Colorbar
            cbar = plt.colorbar(scatter, ax=axes[1])
            cbar.set_label('Trade Size', fontsize=12, fontweight='bold')
            
            axes[1].set_ylabel('Precio ($)', fontsize=13, fontweight='bold')
            axes[1].set_xlabel('Hora (ET - Eastern Time)', fontsize=13, fontweight='bold')
            axes[1].set_title(
                f'{ticker} - MARKET Trades (09:30-20:00 ET) | {date_str} | {len(df_mkt):,} trades',
                fontsize=15, fontweight='bold', pad=20
            )
            axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M', tz=ET))
            axes[1].xaxis.set_major_locator(mdates.HourLocator(interval=1, tz=ET))
            axes[1].grid(True, alpha=0.3, linestyle='--')
            plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
            
            # Línea de precio promedio
            avg_price = df_mkt_pd['price'].mean()
            axes[1].axhline(avg_price, color='orange', linestyle='--', 
                           linewidth=1.5, alpha=0.7, label=f'Avg: ${avg_price:.2f}')
            axes[1].legend(loc='upper left', fontsize=11)
            
            print(f"\n✅ MARKET: {len(df_mkt):,} trades")
            print(f"   Timerange: {df_mkt_pd['timestamp_et'].min()} → {df_mkt_pd['timestamp_et'].max()}")
            print(f"   Price range: ${df_mkt_pd['price'].min():.2f} → ${df_mkt_pd['price'].max():.2f}")
            print(f"   Total volume: {df_mkt_pd['size'].sum():,}")
        else:
            axes[1].text(0.5, 0.5, 'Sin datos market', 
                        ha='center', va='center', fontsize=20, color='gray')
            axes[1].set_title(f'{ticker} - MARKET (sin datos)', fontsize=15, fontweight='bold')
            axes[1].set_xlabel('Hora (ET)', fontsize=13, fontweight='bold')
            print(f"\n⚠️  No market trades")
        
        plt.tight_layout()
        plt.show()
        
        # ================================================================
        # ESTADÍSTICAS RESUMIDAS
        # ================================================================
        print(f"\n{'='*80}")
        print(f"📊 RESUMEN - {ticker} | {date_str}")
        print(f"{'='*80}")
        
        if df_pre is not None and len(df_pre) > 0:
            print(f"\n🌅 PREMARKET:")
            print(f"   Trades: {len(df_pre):,}")
            print(f"   Volume: {df_pre['size'].sum():,}")
            print(f"   Avg trade size: {df_pre['size'].mean():.1f}")
            vwap = (df_pre['price'] * df_pre['size']).sum() / df_pre['size'].sum()
            print(f"   Price VWAP: ${vwap:.2f}")
        
        if df_mkt is not None and len(df_mkt) > 0:
            print(f"\n📈 MARKET:")
            print(f"   Trades: {len(df_mkt):,}")
            print(f"   Volume: {df_mkt['size'].sum():,}")
            print(f"   Avg trade size: {df_mkt['size'].mean():.1f}")
            vwap = (df_mkt['price'] * df_mkt['size']).sum() / df_mkt['size'].sum()
            print(f"   Price VWAP: ${vwap:.2f}")
        
        print(f"\n{'='*80}")
        
        return df_pre, df_mkt
    
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# =====================================================================
# WIDGETS INTERACTIVOS
# =====================================================================

# Obtener tickers disponibles
ticker_dirs = [d.name for d in TRADES_DIR.iterdir() if d.is_dir() and d.name != '_batch_temp']
ticker_dirs = sorted(ticker_dirs)

print("="*80)
print("VISUALIZADOR INTERACTIVO DE TRADES TICK-LEVEL")
print("="*80)
print(f"\n✅ {len(ticker_dirs):,} tickers disponibles")
print(f"✅ Zona horaria: America/New_York (Eastern Time)")
print("\n")

# Crear widgets
ticker_dropdown = widgets.Dropdown(
    options=ticker_dirs,
    description='Ticker:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

date_dropdown = widgets.Dropdown(
    description='Fecha:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

plot_button = widgets.Button(
    description='📊 GRAFICAR TRADES',
    button_style='success',
    layout=widgets.Layout(width='250px', height='40px')
)

output_area = widgets.Output()

# =====================================================================
# LÓGICA DE ACTUALIZACIÓN
# =====================================================================

def update_dates(change):
    """Actualiza las fechas disponibles cuando se selecciona un ticker"""
    ticker = change['new']
    ticker_dir = TRADES_DIR / ticker
    
    # Buscar todas las fechas disponibles
    dates = []
    for year_dir in sorted(ticker_dir.glob("year=*")):
        for month_dir in sorted(year_dir.glob("month=*")):
            for day_dir in sorted(month_dir.glob("day=*")):
                date_str = day_dir.name.replace("day=", "")
                dates.append(date_str)
    
    date_dropdown.options = sorted(dates, reverse=True) if dates else ['Sin datos']
    if dates:
        date_dropdown.value = sorted(dates, reverse=True)[0]  # Fecha más reciente

def plot_selected(button):
    """Ejecuta la visualización cuando se presiona el botón"""
    with output_area:
        clear_output(wait=True)
        
        ticker = ticker_dropdown.value
        date_str = date_dropdown.value
        
        if date_str == 'Sin datos':
            print(f"❌ No hay datos disponibles para {ticker}")
            return
        
        print(f"🚀 Graficando trades tick-level: {ticker} | {date_str}")
        print("="*80)
        print()
        
        plot_trades_analysis(ticker=ticker, date_str=date_str)

# Conectar eventos
ticker_dropdown.observe(update_dates, names='value')
plot_button.on_click(plot_selected)

# Inicializar
if len(ticker_dirs) > 0:
    ticker_dropdown.value = ticker_dirs[0]

# =====================================================================
# MOSTRAR UI
# =====================================================================

print("📋 INSTRUCCIONES:")
print("1. Selecciona un TICKER (actualización automática de fechas)")
print("2. Selecciona una FECHA (ordenadas de más reciente a más antigua)")
print("3. Click en 'GRAFICAR TRADES'")
print("\n✨ Se mostrarán: Premarket + Market trades")
print("⏰ Todas las horas en ET (Eastern Time - Nueva York)")
print("🔵 Premarket: línea azul + puntos | 🔴 Market: línea roja + puntos")
print("📏 Tamaño del punto = tamaño del trade")
print("📈 Línea conecta los ticks para ver el movimiento del precio\n")

display(widgets.VBox([
    widgets.HBox([ticker_dropdown, date_dropdown]),
    plot_button,
    output_area
]))

VISUALIZADOR INTERACTIVO DE TRADES TICK-LEVEL

✅ 766 tickers disponibles
✅ Zona horaria: America/New_York (Eastern Time)


📋 INSTRUCCIONES:
1. Selecciona un TICKER (actualización automática de fechas)
2. Selecciona una FECHA (ordenadas de más reciente a más antigua)
3. Click en 'GRAFICAR TRADES'

✨ Se mostrarán: Premarket + Market trades
⏰ Todas las horas en ET (Eastern Time - Nueva York)
🔵 Premarket: línea azul + puntos | 🔴 Market: línea roja + puntos
📏 Tamaño del punto = tamaño del trade
📈 Línea conecta los ticks para ver el movimiento del precio



: 